In [ ]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels

In [ ]:
dataset_setosa = pd.read_csv("df_setosa", sep=',')
dataset_versicolor = pd.read_csv("df_versicolor", sep=',')
dataset_virginica = pd.read_csv("df_virginica", sep=',')

In [ ]:
df_setosa_train = dataset_setosa.drop('class', axis=1, inplace=False)
df_setosa_class = dataset_setosa["class"]

In [ ]:
df_versicolor_train = dataset_versicolor.drop('class', axis=1, inplace=False)
df_versicolor_class = dataset_versicolor["class"]

In [ ]:
df_virginica_train = dataset_virginica.drop('class', axis=1, inplace=False)
df_virginica_class = dataset_virginica["class"]

In [ ]:
class Perceptron_simples():
    
    # FUNÇÃO __INIT__: primeira função que vai rodar quando eu instanciar a classe
    # Método construtor de objetos. Self receberá a instância criada
    def __init__(self, amostras_entrada, saidas, taxa_aprendizado=0.05, epocas=200, bias=-1):
        self.amostras_entrada = amostras_entrada
        self.saidas = saidas
        self.taxa_aprendizado = taxa_aprendizado
        self.epocas = epocas
        self.bias = bias
        self.num_amostras = len(amostras_entrada) # número de padrões do meu dataset
        self.num_atributos = len(amostras_entrada[0]) # número de colunas do mue dataset
        self.vetor_pesos = []
        
    def treino_rede(self):
        
        # Insiro o valor de bias/viés na posição zero para cada padrão da minha lista de amostras_entrada
        for amostra in self.amostras_entrada:
            amostra.insert(0, self.bias)
            
        # Gera valores aleatórios entre 0 e 1 para compor o vetor de pesos, do mesmo tamanho do número de colunas
        for k in range(self.num_atributos):
            self.vetor_pesos.append(random.random())
        
        # Insere o bias na posição zero do vetor de pesos
        self.vetor_pesos.insert(0, self.bias)
        
        num_epocas = 0 # Inicializa a contagem de épocas
        sum_erros = []
        #sum_epocas = []
        
        while True:
        
            # Inicializa a variável erro, para poder ser testada ao final do loop. Se continuar falso, não houve mais erro
            erro = False
            sum_erros_epoca = 0
            
            for i in range(self.num_amostras):
                u = 0 # Inicializa a função de ativação
                
                for j in range(self.num_atributos + 1): # +1 pelo fato de ter adicionado o valor do bias
                    u += self.vetor_pesos[j] * self.amostras_entrada[i][j] # Executa o somatório WiXi
                    
                y_saida = self.valor_saida(u) # função degrau sobre o valor da função de ativação
                
                if y_saida != self.saidas[i]: 
                    erro_true = self.saidas[i] - y_saida # e = d - y
                    sum_erros_epoca += abs(erro_true)
                    
                    for m in range(self.num_atributos + 1): # se há erro, atualiza o vetor de pesos
                        self.vetor_pesos[m] = self.vetor_pesos[m] + self.taxa_aprendizado * erro_true * self.amostras_entrada[i][m]
                        
                    erro = True # se houve erro, continua true/no loop
            
            sum_erros.append(sum_erros_epoca)
            #sum_epocas.append(num_epocas)
            num_epocas += 1 # incrementa o número de épocas
            
            # Condição de saída: caso erro continue False após o teste com o real ou o numero de epocas estoure
            if not erro or num_epocas >= self.epocas:
                #print(num_epocas)
                break
                
        return sum_erros
        
    def teste_rede(self, new_amostras): # Função para testar novas amostras
        
        new_amostras.insert(0, self.bias)
        u = 0
        
        for i in range(self.num_atributos + 1):
            u += self.vetor_pesos[i] * new_amostras[i]
            
        y_saida = self.valor_saida(u)
        return y_saida
        
        
    def valor_saida(self, u):
        
        if u>0:
            return 1
        return 0

In [ ]:
def data_test(train_data, class_data, test_size, n_iter, class_var):
    
    acc_final = []
    std_final = []
    pred = []
    conj_test = []

    for rep in range(1 , n_iter + 1):
        X_train, X_test, y_train, y_test = train_test_split(train_data,class_data, 
                                                            test_size=test_size, shuffle=True)
        print(X_train)
        print(X_test)
        conj_test.append([X_test])

        amostras = X_train.values.tolist()
        saidas = y_train.values.tolist()
        new_amotras = X_test.values.tolist()
        amostra_teste = y_test.values.tolist()

        rede = Perceptron_simples(amostras, saidas)
        J = rede.treino_rede()
        
        epoch = np.linspace(1,len(J),len(J))
        %matplotlib inline
        plt.figure()
        plt.plot(epoch, J)
        plt.xlabel('Época')
        plt.ylabel('Somatório do Erro')
        plt.title('Gráfico de Convergência - Perceptron Simples')
        plt.show()

        classe_predita = []

        for pd in range(len(new_amotras)):
            y = rede.teste_rede(new_amotras[pd])
            classe_predita.append(y)
            #print("Esperado=%d, Predito=%d" % (amostra_teste[pd], y))

        pred.append([classe_predita])
        plot_confusion_matrix(amostra_teste, classe_predita, classes=['Não-{}'.format(class_var), class_var],
                     title='Matriz de Confusão');
        
        acc = metrics.accuracy_score(amostra_teste, classe_predita)
        std = np.sqrt(abs(metrics.explained_variance_score(amostra_teste, classe_predita)))

        print("Realização {}: Tx. Acerto = {}".format(rep, acc))
        acc_final.append(acc)
        std_final.append(std)

    acc_real = np.mean(acc_final)
    std_real = np.mean(std_final)
    print("\nAcurácia Final = {}, Desvio Padrão Final = {}" .format(acc_real, std_real))
    
    return rede, pred, conj_test

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          title=None,
                          cmap=plt.cm.BuGn):

    # Computa a matriz de confusão
    cm = metrics.confusion_matrix(y_true, y_pred)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Real / Esperado',
           xlabel='Predito')

    # Rotaciona os labels
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    normalize = False
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    
    plt.show()
    return ax

In [ ]:
data_test(df_versicolor_train, df_versicolor_class, test_size=0.25, n_iter=20, class_var='Versicolor')

In [ ]:
data_test(df_virginica_train, df_virginica_class, test_size=0.25, n_iter=20, class_var='Virginica')

In [ ]:
data_test(df_setosa_train, df_setosa_class, test_size=0.25, n_iter=20, class_var='Setosa')

# REALIZAÇÕES - CONJUNTO DE DADOS ARTIFICIAL 1

In [ ]:
dataset_art_1 = pd.read_csv("artificial_1.csv", sep=';')
dataset_art_1.head()

In [ ]:
df_art_train = dataset_art_1.drop('classe', axis=1, inplace=False)
df_art_class = dataset_art_1["classe"]

In [ ]:
df_art_train.head()

In [ ]:
set(df_art_class)

In [ ]:
model, pred, cj_te = data_test(df_art_train, df_art_class, test_size=0.3, n_iter=20, class_var='Zero')

# SUPERFÍCIE DE DECISÃO

In [ ]:
def decision_surface(pred_points, conj_te, n_rep):
    
    pred_pt = pred_points[n_rep][0]
    
    plt_x = conj_te[n_rep][0].values[:,0]
    plt_y = conj_te[n_rep][0].values[:,1]
    
    plt.scatter(plt_x, plt_y, c=pred[n_rep][0], marker='s', s=100)
    plt.xticks(np.arange(-0.3, 1.3, step=0.3))
    plt.yticks(np.arange(-0.3, 1.3, step=0.3))
    plt.xlabel("X1 Points", fontsize=18);
    plt.ylabel("X2 Points", fontsize=18);
    plt.grid()
    plt.title("Superfície de Decisão - Realização {}".format(n_rep + 1), fontsize=18);

In [ ]:
%matplotlib inline
decision_surface(pred, cj_te, n_rep=10)

In [ ]:
classe_predita = []

for i in np.arange(0,1.05,0.05):
    for k in np.arange(0,1.05,0.05):
        par = [i,k]
        y = model.teste_rede(par)
        classe_predita.append([par[1], par[2], y])

In [ ]:
x1 = []
x2 = []
cl = []

for i in range(0, len(classe_predita)):
    plt_x = classe_predita[i][0]
    plt_y = classe_predita[i][1]
    classe = classe_predita[i][2]
    x1.append(plt_x)
    x2.append(plt_y)
    cl.append(classe)

In [ ]:
plt.scatter(x1, x2, c=cl, marker='s', s=30)
plt.xticks(np.arange(-0.1, 1.2, step=0.2))
plt.yticks(np.arange(-0.1, 1.2, step=0.2))
plt.xlabel("X1 Points", fontsize=18);
plt.ylabel("X2 Points", fontsize=18);
plt.title("Superfície de Decisão - Conjunto Geral", fontsize=17);